In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""

from eos_database import *
from compressor_class import *
from compression import *
from gc_eos_soave import *
from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from viscosity import *
from plenum_system import *
import pdb

In [2]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)




In [ ]:
n_x = 3
n_z = 11

def system_residuals(y, u0, plenum_sys):
    # Separa as variáveis
    x = y[:n_x]
    z = y[n_x:]
    
    # Substitua as expressões abaixo pelas suas equações do modelo em estado estacionário.
    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x, z, u0)
    
    res_ode = np.array([ode_sym[i].item() for i in range(n_x)])
    
    # Calcula os resíduos das equações algébricas
    res_alg = np.array([alg_sym[i] for i in range(n_z)])

    res = np.concatenate((res_ode, res_alg))
    return res

def compute_steady_state(u0, plenum_sys, x0, z0):
    # Vetor inicial concatenado
    y0 = np.array(x0 + z0)
    
    # Chama o fsolve para encontrar os zeros da função de resíduos
    sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))
    
    # Separa a solução em x e z
    x_ss = sol[:n_x]
    z_ss = sol[n_x:]
    return x_ss, z_ss

if __name__ == '__main__':
    Phi, eta, Mach, Gimp, G2, Gdif, PHI, G2s, k = compression.character(compressor, m = 10, N = 750, Gi_1 = gas)
    x0 = [10, G2.T, G2.V]
    # E z0 pode ser (conforme seu código original):
    z0 = [G2.P, G2.P, Gimp.T, Gimp.V,
          Gdif.T, Gdif.V, G2s.T, G2s.V,
          G2.T, G2.V]+ [gas.V]
    u0 = [4500, 300, 750, (0.65), 5000]
    
    # Calcula os estados estacionários
    x_ss, z_ss = compute_steady_state(u0, plenum_sys, x0, z0)
    print("Estado estacionário (variáveis diferenciais):", x_ss)
    print("Estado estacionário (variáveis algébricas):", z_ss)

/tmp/ipykernel_25735/3159779489.py:39: RuntimeWarning: invalid value encountered in scalar power
  u0 = [4500, 300, 750, (10/0.38/(G2.P - 5000)**0.5), 5000]


Estado estacionário (variáveis diferenciais): [1.00000000e+01 3.00000000e+02 1.72647959e+06]
Estado estacionário (variáveis algébricas): [1.44475425e-03 1.44475425e-03 3.00000000e+02 3.50244769e-01
 3.00000000e+02 3.54788964e-01 3.00000000e+02 1.64897037e+06
 3.00000000e+02 1.72647959e+06 5.14917237e-01]


/tmp/ipykernel_25735/3159779489.py:25: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))


In [ ]:
def test_evaluate_dae(plenum_sys, x_test, z_test, u0):# Parâmetros de entrada

    # Avaliação da função
    ode_values, alg_values = plenum_sys.evaluate_dae(None, x_ss, z_ss, u0)

    # Imprimindo os resultados
    print("Valores das ODEs:")
    for i, val in enumerate(ode_values):
        print(f"ODE {i}: {val}")

    print("\nValores das equações algébricas:")
    for i, val in enumerate(alg_values):
        print(f"Algebrica {i}: {val}")
    return ode_values, alg_values
ode_values, alg_values = test_evaluate_dae(plenum_sys, x_ss, z_ss, u0)

In [ ]:
def run_simulation(x0, z0, plenum_sys):
    u0 = [4500, 300, 750, (10/0.38/(G2.P - 5000)**0.5), 5000]
    print("\n--- Valores iniciais ---")
    print("x0:", x0)
    print("z0:", z0)
    print("u0:", u0)

    # Verificação básica de NaN/Inf
    if any(np.isnan(x0)) or any(np.isinf(x0)):
        raise ValueError("Valores inválidos em x0 (NaN/Inf)")
    if any(np.isnan(z0)) or any(np.isinf(z0)):
        raise ValueError("Valores inválidos em z0 (NaN/Inf)")
    if any(np.isnan(u0)) or any(np.isinf(u0)):
        raise ValueError("Valores inválidos em u0 (NaN/Inf)")

    # Definição dos símbolos
    x_sym = SX.sym('x', 3)
    z_sym = SX.sym('z', 11)
    u_sym = SX.sym('u', 5)

    # Avaliação das equações
    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x_sym, z_sym, u_sym)

    # Funções para avaliação separada
    ode_fun = ca.Function('ode_fun', [x_sym, z_sym, u_sym], ode_sym)
    alg_fun = ca.Function('alg_fun', [x_sym, z_sym, u_sym], alg_sym)

    print("\n--- Avaliação das equações ODE ---")
    ode_val = ode_fun(x0, z0, u0)
    print("Valores ODE:", ode_val)
    
    print("\n--- Avaliação das equações algébricas ---")
    alg_val = alg_fun(x0, z0, u0)
    print("Valores ALG:", alg_val)

    # Verificação de consistência algébrica
    alg_error = np.linalg.norm(alg_val)
    print("\n--- Erro nas equações algébricas ---")
    print("Norma do erro:", alg_error)
    if alg_error > 1e-6:
        print("AVISO: Condições iniciais não satisfazem equações algébricas!")

    # Verificação de valores problemáticos
    def check_nan_inf(arr, name):
        if np.any(np.isnan(arr)) or np.any(np.isinf(arr)):
            bad_idx = np.where(np.isnan(arr) | np.isinf(arr))[0]
            raise ValueError(f"Valores inválidos ({name}) nos índices {bad_idx}")

    check_nan_inf(ode_val, "ODE")
    check_nan_inf(alg_val, "ALG")

    # Criação do DAE e integrador (mantido para referência)
    dae = {
        'x': x_sym,
        'z': z_sym,
        'p': u_sym,
        'ode': vertcat(*ode_sym),
        'alg': vertcat(*alg_sym)
    }

    print("\n--- Iniciando integração ---")
    try:
        integrator_solver = integrator('F', 'idas', dae, 0, 100)
        res = integrator_solver(x0=x0, z0=z0, p=u0)
        print("Integração concluída com sucesso!")
    except Exception as e:
        print("\n--- Erro durante a integração ---")
        print("Tipo do erro:", type(e).__name__)
        print("Mensagem:", str(e))
        raise

    return res

run_simulation(x_ss,z_ss, plenum_sys)